<a href="https://colab.research.google.com/github/steve-keys/Assignment---1--Intro-to-Programming/blob/main/Copy_of_michaels_sandbox_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Algorithm

1. User input ticker symbol
2. Use ticker to get profile page
3. Copied the table
4. For each row of the table
  * Process a row to extract pay
  * Find the last character to determin multiplier
  * Calculate salary, pay * mulitplier
  * Calculate the income tax


Note:  
1. Need to validate the ticker, but for now I will assume the user type in a correct ticker
2. Still need to work out where I save the income tax to

In [ ]:
# Usually run of Google Coolab, so install unique packages
!pip install kora -q

#load packages
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd

     |████████████████████████████████| 57 kB 1.2 MB/s 
     |████████████████████████████████| 56 kB 1.6 MB/s 


In [ ]:
def build_url():
  # User input ticker symbol
  ticker_symbol = input("Please input ticker symbol ")
  # print(ticker_symbol)
  example_url = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
  result = example_url.format(ticker_symbol, ticker_symbol)
  return result



In [ ]:
def get_first_3_cells(row):
  cells = row.find_all('td')
  name = cells[0].text
  title = cells[1].text
  pay = cells[2].text
  return {'name': name, 'title': title, 'pay': pay}


In [ ]:
def get_exec_records(soup):
  table = soup.find('table')
  body = table.find('tbody')
  rows = body.find_all('tr')
  
  exec_records = []
  for row in rows:
    result = get_first_3_cells(row)
    exec_records.append(result)
  return exec_records

In [ ]:
def get_salary(rec):
  pay = rec['pay']
  last_char = pay[-1]
  num_part = pay[0:-1]

  if last_char == 'M':
    salary = float(num_part) * 1000000
    #print("Millions")
  elif last_char == 'k':
    salary = float(num_part) * 1000
    #print("Thousand")
  else:
    salary = 0
    #print("Not sure")

  return salary



In [ ]:
def calculate_tax(income):
  if income <= 18200:
    tax_payable = 0
  elif 18200 < income <= 45000:
    tax_payable = 0.19 * (income - 18200)
  elif 45000 < income < 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)
  elif 120001 < income < 180000:
    tax_payable = 29467 + 0.37 * (income - 120000)
  elif income > 180000:
    tax_payable = 51667 + 0.45 * (income - 180000)


  return tax_payable

In [ ]:
# Get URL
url = build_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source, "html.parser")
records = get_exec_records(soup)

for record in records:
  result = get_salary(record)
  income_tax = calculate_tax(result)
  record['tax'] = income_tax

# creat a pandas datafrme (because recall in a earlier notebok a data frame is easy to save as csv)

execs_df = pd.DataFrame.from_records(records)
execs_df

Please input ticker symbol nab.ax


,name,title,pay,tax
0,"Mr. Ross Maxwell McEwan CBE, B.B.S.","Group CEO, MD & Director",2.54M,1113667.0
1,"Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons)...",Group Chief Financial Officer,1.13M,479167.0
2,Mr. Patrick F. Wright,Group Exec. of Technology & Enterprise Operations,1.72M,744667.0
3,Ms. Sharon J. Cook,Group Exec. of Legal & Commercial Services,912.98k,381508.0
4,Ms. Susan Ferrier,Group Exec. of People & Culture,929.27k,388838.5
5,Ms. Angela Mentis B.Bus,"Group Chief Digital, Data & Analytics Officer",1.62M,699667.0
6,Mr. Shaun Dooley,Chief Risk Officer,1.05M,443167.0
7,Mr. Andrew Irvine,Chief Exec. of Bus. Private Banking Division,333.27k,120638.5
8,Mr. Lesile D. Matheson,Group Chief Operating Officer,N/A,0.0
9,Ms. Sally Mihell,Head of Investor Relations,N/A,0.0
